In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
def parser_image_preds(single_image):
    image_preds = ""
    prediction = single_image.split("seconds.")[1]
    individual_preds_in_image = prediction.split("wheathead")[1:]
    
    for i in individual_preds_in_image:
        pred = i.split(":")[1]
        conf = float("".join(pred.split('\t')[0]).strip().replace("%",""))/100
        
        ny = "".join(i.split(":")[2:]).replace(")","").split(" ")
        bbox = []
        for inner in ny:
            if inner.lstrip('-').isdigit():
                bbox.append(int(inner))
        
        string_to_add = str(str(conf) + " " + str(bbox).replace("[", "").replace("]", "").replace(",", ""))
        image_preds += " " + string_to_add
    return image_preds


## Notebook compiles darknet from scratch, uses weights that I have trained on my machine, and creates submission file using yolov4 on darknet.

In [ ]:
cp -r /kaggle/input/darknet-jpo2/* ./

In [ ]:
os.chdir("/kaggle/working/darknet_jpo/darknet/")

In [ ]:
!ls 

In [ ]:
!cp ../../yolov4-wheat100_best_v2.weights backup/wheat100/

In [ ]:
!ls backup/wheat100/

In [ ]:
!sed '1 s/^.*$/GPU=0/; 2 s/^.*$/CUDNN=0/; 3 s/^.*$/CUDNN_HALF=0/; 5 s/^.*$/AVX=0/; 6 s/^.*$/OPENMP=0/; 7 s/^.*$/LIBSO=0/' -i Makefile

In [ ]:
!head Makefile

In [ ]:
! make clean

In [ ]:
!make 

### Get list of test ids

In [ ]:
test_subs = pd.read_csv("/kaggle/input/global-wheat-detection/sample_submission.csv")
test_image_ids = test_subs.image_id.values
test_subs.head()

In [ ]:
test_image_ids

In [ ]:
from pathlib import Path
import subprocess

root_image = Path("/kaggle/input/global-wheat-detection/test")
test_images = [root_image / f"{img}.jpg" for img in test_subs.image_id]

In [ ]:
with open("test_predict.txt", 'a') as file: # append lines to file
    for i in test_image_ids:
        new_name =  "/kaggle/input/global-wheat-detection/test/{}.jpg".format(i)
        new_name = new_name + "\n"
        file.write(new_name)

In [ ]:
cmd = "./darknet detector test data/wheat100.data cfg/yolov4-wheat100.cfg backup/wheat100/yolov4-wheat100_best_v2.weights -dont_show -ext_output < test_predict.txt > pr_res.txt -thresh 0.25"
b = subprocess.getoutput(cmd)

In [ ]:
!head pr_res.txt

In [ ]:
conf_result_file = "pr_res.txt"
file_l = open(conf_result_file, 'r')   
lines = file_l.read().split('\n')
content_str = "".join(lines)
int_unique_predictions = content_str.split('Enter')[1:]

In [ ]:
res_lines = []  
#single_image = int_unique_predictions[0]
for single_image in int_unique_predictions[:-1]:
    img_id = "".join([i for i in single_image.split(" ") if '.jpg' in i]).replace(":","").split("/")[-1].replace(".jpg", "")
    print(img_id)
    xx = parser_image_preds(single_image)
    #print(img_id, xx)
    res_lines.append({'image_id': img_id, 'PredictionString': xx})

submission_df = pd.DataFrame(res_lines)     

In [ ]:
submission_df.head()

In [ ]:
os.chdir("/kaggle/working/")

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
!ls

In [ ]:
!rm -r darknet